In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

import matplotlib.pyplot as plt



training_data_generator = ImageDataGenerator(rescale = 1/255, zoom_range = 0.2, rotation_range = 15, width_shift_range = 0.05, height_shift_range = 0.05)
validation_data_generator = ImageDataGenerator(rescale = 1/255, validation_split = 0.2)

validation_iterator = validation_data_generator.flow_from_directory(directory = r"C:\Users\super\Downloads\Covid19-dataset\test", class_mode = 'categorical', color_mode = 'grayscale', target_size = (256,256), batch_size = 32, subset = 'training' )
testing_iterator = validation_data_generator.flow_from_directory(directory = r"C:\Users\super\Downloads\Covid19-dataset\test", class_mode = 'categorical', color_mode = 'grayscale', target_size = (256,256), batch_size = 32, subset = 'validation' )
training_iterator = training_data_generator.flow_from_directory(directory = r"C:\Users\super\Downloads\Covid19-dataset\train",  class_mode = 'categorical', color_mode = 'grayscale', target_size = (256,256), batch_size = 32)

sample_batch_input, sample_batch_labels = training_iterator.next()
validation_batch_input, validation_batch_labels = validation_iterator.next()
print(sample_batch_input.shape, sample_batch_labels.shape)

model = Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(256,256,1)))
model.add(tf.keras.layers.Conv2D(5, 5, strides = 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
model.add(layers.Dropout(0.1))
model.add(tf.keras.layers.Conv2D(3, 3, strides = 1, activation = 'relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'valid'))
model.add(layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(3, 3, strides=1, activation="relu"))
model.add(tf.keras.layers.Conv2D(1, 1, strides=1, activation="relu"))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation = 'softmax'))
model.summary()

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=20)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=.001), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()],)
model.fit(
        training_iterator,
        steps_per_epoch=training_iterator.samples/32, epochs=5,
        validation_data=validation_iterator,
        validation_steps=validation_iterator.samples/32,
        callbacks=[es])

predictions = model.predict(testing_iterator, batch_size = 32)  
true_classes = testing_iterator.classes
predicted_classes = np.argmax(predictions, axis = 1)
print(predicted_classes, true_classes)



Found 53 images belonging to 3 classes.
Found 13 images belonging to 3 classes.
Found 251 images belonging to 3 classes.
(32, 256, 256, 1) (32, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 84, 84, 5)         130       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 5)         0         
_________________________________________________________________
dropout (Dropout)            (None, 42, 42, 5)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 3)         138       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 3)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 2